In [ ]:
import cv2 as cv
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import gdown


In [ ]:
#telechargement de la base et autres dossiers
url = "https://drive.google.com/uc?id=1VrSh6miJhkHcR1jmaP3gMCp8KYDYtqvE"  
output = "VideosTest.zip"
gdown.download(url, output, quiet=False)
#unzip file
!unzip "VideosTest.zip"

In [ ]:
#Calcul du Flot Optique Dense par la méthode de Farneback


#Le flux vidéo est lu comme des captures d'objets par VideoCapture
cap = cv.VideoCapture("/content/walking.mp4")
# ret retourne une valeur bouléenne indiquant l'obtention d'une capture, first_frame est le premier frame dans la séquence vidéo
ret, first_frame = cap.read()
# on converti ensuite le frame en niveau de gris car nous avons juste besoin de la luminance pour la détection des contours, ainsi on économise en puissance de calcul
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
# On crée ensuite une image avec une intensité nulle ayant la meme dimention que les frames de la vidéo.
mask = np.zeros_like(first_frame)
# on règle la saturation de l'image au maximum
mask[..., 1] = 255

 
while(cap.isOpened()):
    #ret retourne une valeur bouléenne indiquant l'obtention d'un frame, frame est le frame courant capturé dans la vidéo.
    ret, frame = cap.read()
    #on converti chaque frame nouvellement capturé en niveau de gris 
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
   
    #Calcul du flow optique dense par la méthode de Farneback.
    flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
   
    #Calcul des normes des angles des vecteurs 2D caratéristiques du flot optique
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
    # Définit la teinte de l'image en fonction de la direction du flot optique
    mask[..., 0] = angle * 180 / np.pi / 2
    # Définit la valeur de l'image en fonction de l'amplitude du flux optique (normalisé)
    mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
    
    # Conversion de l'espace couleur HSV à l'espace RGB(BGR)
    rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)
    
    #on recupère la hauteur et la largeur du frame capturé pour définir la dimension d'affichage
    h, w, _ = np.array(frame).shape
    valeur_h = 0.6
    valeur_w = 0.4
    
    # on éffectue une concaténation entre le frame d'origine et l'image résultant du calcul du flot optique et on les redimensionne pour les afficher sur la meme ligne.
    affich_associe = np.concatenate((cv.resize(frame, (int(h * valeur_h), int(w * valeur_w))),
                                  cv.resize(rgb, (int(h * valeur_h), int(w * valeur_w)))), axis=1)

    # On affiche ensuite le resultat obtenu
    print('flow optic dense')
    cv2_imshow(affich_associe)
    # mise à jour du frame précédent
    prev_gray = gray
    # Les frames sont lu à des intervalles de 30 millisecondes. 
    if cv.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

In [ ]:
#Estimation du flot optique en fonction des normes
 
# Le flux vidéo est lu comme des captures d'objets par VideoCapture
cap = cv.VideoCapture("/content/walking.mp4")
# ret retourne une valeur bouléenne indiquant l'obtention d'une capture, first_frame est le premier frame dans la séquence vidéo
ret, first_frame = cap.read()
# on converti ensuite le frame en niveau de gris car nous avons juste besoin de la luminance pour la détection des contours, ainsi on économise en puissance de calcul
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
# On crée ensuite une image avec une intensité nulle ayant la meme dimention que les frames de la vidéo.
mask = np.zeros_like(first_frame)
# on règle la saturation et la teinte de l'image à zéro
mask[..., 1] = 0
mask[..., 0] = 0
 
while(cap.isOpened()):
    #ret retourne une valeur bouléenne indiquant l'obtention d'un frame, frame est le frame courant capturé dans la vidéo.
    ret, frame = cap.read()
    # on converti chaque frame nouvellement capturé en niveau de gris 
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calcul du flow optique dense par la méthode de Farneback.    
    flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
   
    #Calcul des normes et des angles des vecteurs 2D caratéristiques du flot optique
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])

    #On Définit la valeur d'un seul canal de l'image en fonction de l'amplitude du flux optique (normalisé), la saturation et la teinte restant à zéro
    mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
    
    #Conversion de l'espace couleur HSV à l'espace RGB(BGR)
    rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)


    #on recupère la hauteur et la largeur du frame capturé pour définir la dimension d'affichage
    h, w, _ = np.array(frame).shape
    valeur_h = 0.6
    valeur_w = 0.4
    
    # on éffectue une concaténation entre le frame d'origine et l'image résultant du calcul du flot optique et on les redimensionne pour les afficher sur la meme ligne.
    affich_associe = np.concatenate((cv.resize(frame, (int(h * valeur_h), int(w * valeur_w))),
                                  cv.resize(rgb, (int(h * valeur_h), int(w * valeur_w)))), axis=1)

    
    # On affiche ensuite le resultat obtenu
    print("flot optique basé sur les normes")
    cv2_imshow(affich_associe)
    #mise à jour du frame précédent
    prev_gray = gray
    # Les frames sont lu à des intervalles de 30 millisecondes
    if cv.waitKey(30) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

In [ ]:
#Segmentation avec seuillage selon les normes de flot optique des objets bougeant plus rapidement
 
# Le flux vidéo est lu comme des captures d'objets par VideoCapture
cap = cv.VideoCapture("/content/walking.mp4")
# ret retourne une valeur bouléenne indiquant l'obtention d'une capture, first_frame est le premier frame dans la séquence vidéo
ret, first_frame = cap.read()
# on converti ensuite le frame en niveau de gris car nous avons juste besoin de la luminance pour la détection des contours, ainsi on économise en puissance de calcul
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
# On crée ensuite une image avec une intensité nulle ayant la meme dimention que les frames de la vidéo.
mask = np.zeros_like(first_frame)
# on règle la saturation et la teinte de l'image au maximum
mask[..., 1] = 255
mask[..., 0] = 255
#on défini le seuil et la taille du filtre
seuil = 40
kernel = np.ones(7)
 
while(cap.isOpened()):
    #ret retourne une valeur bouléenne indiquant l'obtention d'un frame, frame est le frame courant capturé dans la vidéo.
    ret, frame = cap.read()
    #on converti chaque frame nouvellement capturé en niveau de gris 
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
   
    #Calcul du flow optique dense par la méthode de Farneback.
    flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
   
     #Calcul des normes et des angles des vecteurs 2D caratéristiques du flot optique
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
    
    #On Définit la valeur d'un seul canal de l'image en fonction de l'amplitude du flux optique (normalisé), la saturation et la teinte restant à la valeur maximum
    mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
    mask[..., 0] = 255
    #on affecte au canal de la teinte les valeurs obtenu de la segmentation suivant les valeurs des normes des vecteurs flot optiques.
    ret, mask[..., 0] = cv.threshold(mask[..., 2], seuil, 255, cv.THRESH_BINARY)
    #Conversion de l'espace couleur HSV à l'espace RGB(BGR)
    rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)

    
    #on effectue une opération de opening(érosion suivie d’une dilatation ) sur le canal contenant le résultat de la segmentation
    opening = cv.morphologyEx(mask[..., 0], cv.MORPH_OPEN, kernel)  
    #on effectue une opération de closing(dilatation suivie de l'érosion) sur le resulat du opening obtenu
    closing = cv.morphologyEx(opening, cv.MORPH_CLOSE, kernel)  


     #On crée donc un masque avec les résultats de la segmentation obtenus précédemment
    masque = cv.cvtColor(closing, cv.COLOR_GRAY2BGR)
     #on fait une concaténation entre l'image originale et l'image segmenté en rgb
    affich_1 = np.concatenate((frame, rgb), axis=1)
    #on fait une concaténation entre l'image originale et le masque de la segmentation
    affich_2 = np.concatenate((frame, masque), axis=1)
    #on associe les deux lignes précédentes et on les affiches
    affich_final = np.concatenate((affich_1, affich_2), axis=0)
  
    
    #On affiche ensuite le resultat obtenu
    cv2_imshow(affich_final)
    #mise à jour du frame précédent
    prev_gray = gray
    #Les frames sont lu à des intervalles de 30 millisecondes
    if cv.waitKey(30) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

In [ ]:
#Segmentation avec seuillage selon les normes de flot optique des objets bougeant à différentes vitesses

#Le flux vidéo est lu comme des captures d'objets par VideoCapture
cap = cv.VideoCapture("/content/walking.mp4")
#ret retourne une valeur bouléenne indiquant l'obtention d'une capture, first_frame est le premier frame dans la séquence vidéo
ret, first_frame = cap.read()

shape = (first_frame.shape[1], first_frame.shape[0])
#on converti ensuite le frame en niveau de gris car nous avons juste besoin de la luminance pour la détection des contours, ainsi on économise en puissance de calcul
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
#On crée ensuite une image avec une intensité nulle ayant la meme dimention que les frames de la vidéo.
mask = np.zeros_like(first_frame)

# on définit deux valeur de seuils et la taille du filtre
seuil_1 = 40
seuil_2 = 100
kernel = np.ones(7)
 
while(cap.isOpened()):
    #ret retourne une valeur bouléenne indiquant l'obtention d'un frame, frame est le frame courant capturé dans la vidéo.
    ret, frame = cap.read()
    #on converti chaque frame nouvellement capturé en niveau de gris
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
   
    #Calcul du flow optique dense par la méthode de Farneback.
    flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    #Calcul des normes et des angles des vecteurs 2D caratéristiques du flot optique
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
    
    #On Définit la valeur d'un seul canal de l'image en fonction de l'amplitude du flux optique (normalisé), la saturation et la teinte restant à la valeur maximum
    magnitude = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
    
    #on règle la saturation et le canal des valeurs à 128 et la teinte de l'image à zéro
    mask[..., 0] = 0
    mask[..., 1] = 128
    mask[..., 2] = 128

    #on applique le seuil_2 pour une segmentation sur les canaux de teinte et de saturation
    ret, mask[..., 0] = cv.threshold(magnitude, seuil_2, 255, cv.THRESH_BINARY)
    ret, mask[..., 1] = cv.threshold(magnitude, seuil_2, 255, cv.THRESH_BINARY)
    #on parcours l'ensemble de pixels du canal de saturation et on donne la valeur 128 aux pixels valant 0 après le seuillage
    mask[..., 1] = np.array(
        [128 if value == 0 else value for value in mask[..., 1].reshape(shape[0] * shape[1])]).reshape(shape[1],shape[0])
     #on applique le seuil_1 pour une segmentation sur les canaux de teinte et de value                                                                                                shape[0])
    ret, mask[..., 0] = cv.threshold(magnitude, seuil_1, 255, cv.THRESH_BINARY)
    ret, mask[..., 2] = cv.threshold(magnitude, seuil_1, 255, cv.THRESH_BINARY)
    #on parcours l'ensemble des pixels du canal des value et on donne la valeur 128 aux pixels valant 0 après le seuillage
    mask[..., 2] = np.array(
        [128 if value == 0 else value for value in mask[..., 2].reshape(shape[0] * shape[1])]).reshape(shape[1], shape[0])
                                                                                                      


    #Conversion de l'espace couleur HSV à l'espace RGB(BGR)
    rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)

    #on effectue une opération de opening(érosion suivie d’une dilatation ) sur le canal de saturation contenant le resultat du seuillage avec seuil_2
    opening_1 = cv.morphologyEx(mask[..., 1], cv.MORPH_OPEN, kernel)  
    #on effectue une opération de closing(dilatation suivie de l'érosion) sur le resulat du opening obtenu
    closing_1 = cv.morphologyEx(opening_1, cv.MORPH_CLOSE, kernel) 
    
    #on effectue une opération de opening(érosion suivie d’une dilatation ) sur le canal de valeur contenant le resultat du seuillage avec seuil_1
    opening_2 = cv.morphologyEx(mask[..., 2], cv.MORPH_OPEN, kernel) 
    #on effectue une opération de closing(dilatation suivie de l'érosion) sur le resulat du opening obtenu
    closing_2 = cv.morphologyEx(opening_2, cv.MORPH_CLOSE, kernel) 


    #On crée donc un masque avec les résultats de la segmentation obtenus précédemment en associant les deux masques obtenus des deux seuillage
    masque = closing_1 + closing_2
    masque = cv.cvtColor(masque, cv.COLOR_GRAY2BGR)

    #on fait une concaténation entre l'image originale et l'image segmenté en rgb
    affich_1 = np.concatenate((frame, rgb), axis=1)
    #on fait une concaténation entre l'image originale et le masque de la segmentation
    affich_2 = np.concatenate((frame, masque), axis=1)
    #on associe les deux lignes précédentes et on les affiches
    affich_associe = np.concatenate((affich_1, affich_2), axis=0)

    cv2_imshow(affich_associe)
    
    prev_gray = gray
    
    if cv.waitKey(30) & 0xFF == ord('q'):
        break
        
cap.release()
cv.destroyAllWindows()